<a href="https://colab.research.google.com/github/basile-bron/Title-clickbait-detection/blob/NLP_1.0/ClickBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/basile-bron/Title-clickbait-detection.git

Cloning into 'Title-clickbait-detection'...
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 303 (delta 18), reused 22 (delta 7), pack-reused 256
Receiving objects: 100% (303/303), 41.38 MiB | 11.97 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [2]:
!pip install tensorflow==2.5.0
!pip install -U tensorflow-text==2.5.0
!pip install transformers==4.8.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.4/462.4 KB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 69.6 MB/s eta 0:00:00
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=495138291812b3d83dbe806c1f1a6f5e37ad537cd49a071cc31c9273a001a81d
  Stored in directory: /root/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-linux_x86_64.whl

In [3]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.dpi'] = 200

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.5.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [26]:
# Load dataset from URL
df = pd.read_csv("https://raw.githubusercontent.com/basile-bron/Title-clickbait-detection/NLP_1.0/data/titles.csv")


# apply the conversion function to the column containing the scores
df['total'] = df['total'].apply(lambda x: x/100)



In [33]:
# Split dataset into pre-specified training, validation, and test sets 
from sklearn.model_selection import train_test_split

#cleaning  taking the rows where "titles" is not NA:
df = df[df['titles'].notna()]

# specify the columns you want to use as features and labels
X = df.drop(['total','category1____001','ratings','deleted','id'], axis=1)
y = df['total']

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# split again the X_train to X_train and X_val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("X_train head:")
print(X_train.head())

print("y_train head:")
print(y_train.head())

X_train head:
                                                 titles
5133  PlayStation Backwards Compatibility | Document...
5386  Imaqtpie - WHY DOES EVERYONE HATE LEAGUE OF LE...
4741                                           - Lyrics
2554     PUTIN JUST EXPOSED THE PLOT TO DESTROY AMERICA
4596     Gegenseitige Lage von Geraden im Raum - REMAKE
y_train head:
5133    0.00
5386    0.25
4741    0.00
2554    1.00
4596    0.00
Name: total, dtype: float64


In [28]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [29]:
model_name = 'bert-base-cased'
BERT_tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                               model_max_length = 128) # by default, BERT takes inputs up to 512, but that will take a very long time to train

In [34]:
X_train = BERT_tokenizer(list(X_train['titles']), padding='max_length', truncation=True, return_tensors='tf')
X_test = BERT_tokenizer(list(X_test['titles']), padding='max_length', truncation=True, return_tensors='tf')
X_val = BERT_tokenizer(list(X_val['titles']),padding='max_length', truncation=True, return_tensors='tf')


In [35]:
# download pretrained model as TF, specify that there is one label in output layer
BERT_model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 1) 

BERT_model.summary()

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  769       
Total params: 108,311,041
Trainable params: 108,311,041
Non-trainable params: 0
_________________________________________________________________


In [36]:
BERT_model.compile(optimizer=tf.keras.optimizers.Adam(5e-5), #still adam optimizer, but we want to specify a small initial learning rate
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [37]:
history = BERT_model.fit(dict(X_train), y_train, 
                         validation_data=(dict(X_val), y_val), 
                         epochs = 2, 
                         batch_size = 16) 

# don't worry about all the warning messages--the Transformers package is extremely verbose on TensorFlow for some reason.

Epoch 1/2


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


 11/222 [>.............................] - ETA: 1:11:59 - loss: 0.6405 - accuracy: 0.5114

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()